In [ ]:
#IMPORT PACKAGES
import os
import csv
import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options




In [ ]:
#IMPORT FUNCTIONS HERE

# DEBUGGING

## Selenium SALARY Lookup

In [ ]:
#input_year_test = input_match_date
#natl_test = input_nationality

# transfermarkt_filtered_result_AAA = 'Mamadou N\'diaye'

#debug - get URL for selenium lookup capology
yearstr = input_year_test.split(".")[2]
full_num = '20' + yearstr
full_num = int(full_num)
#for Ndiaye case 
#search = f"{transfermarkt_filtered_result_AAA} {natl_test} capology {full_num}"

#Zelarayan case
search = f"{name_match[2]} {natl_test} capology {full_num}"

url = 'https://www.google.com/search'

headers = {'Accept' : '*/*', 'Accept-Language': 'en-US,en;q=0.5','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
}
parameters = {'q': search}

content = requests.get(url, headers = headers, params = parameters).text
soup = BeautifulSoup(content, 'html.parser')

search_content = soup.find(id = 'search')
first_link = search_content.find('a')

url_tosplit = (first_link['href'])

###NEW from the if statement down 
if(compareNames(name_match[2], url_tosplit)): #transfermarkt_filtered_result_AAA
    driver = webdriver.Chrome()
    driver.get(url_tosplit)

    # Wait for elements to be present (adjust timeout as needed)
    wait = WebDriverWait(driver, 10)
    
    # Add a retry loop to handle the case where element_text_string is empty
    max_retries = 3
    current_retry = 0
    while current_retry < max_retries:
        elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*")))
        
        element_num = 0
        element_text_string = []
        for element in elements:
            
            if element_num == 0:
                try:
                    text = element.text

                    # Check if the element contains both "$" signs and "-" characters
                    if "$" in text and "-" in text:
                        element_num += 1
                        element_text_string.append(text)

                except Exception as e:
                    print(f"Error retrieving text: {e}")

        driver.quit()

        # Check if element_text_string is not empty after retry
        if element_text_string:
            break
        else:
            current_retry += 1
            print(f"Retrying to retrieve text. Retry {current_retry}/{max_retries}")

else:
    #print('first URL not right')
    # Extract and print text of all links on the first page of Google
    # Extract and print text of all links on the first page of Google
    # Extract and print text of all links on the first page of Google
    all_links = search.find_all('a')
    
    # Counter to keep track of matches found
    match_counter = 0
    name_to_find = remove_apostrophes_backticks_single_string(transfermarkt_filtered_result_AAA)
    
    for link in all_links:
        try:
            text = link.find('h3').text  # Assuming the result title is wrapped in an 'h3' tag
            url = link.get('href')
            #print(f"Result: {text}\nURL: {url}\n")

            # Check if name_to_find or transfermarkt_filtered_result_AAA is present in the link
            if name_to_find.lower() in url.lower() or "transfermarkt_filtered_result_AAA" in url:
                print("Match found!", name_to_find, url)
                match_counter += 1

        except AttributeError:
            pass  # Handle cases where 'h3' tag is not found or has no text

    # Check if no matches were found
    if match_counter == 0:
        print("No matches found online.")
        #return 0 
        #return 0

In [ ]:
#old "if comparenames is true" bit

if(compareNames(transfermarkt_filtered_result_AAA, url_tosplit)): #name_match[2]
    driver = webdriver.Chrome()
    driver.get(url_tosplit)

    # Wait for elements to be present (adjust timeout as needed)
    wait = WebDriverWait(driver, 10)
    elements = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//*")))

    element_num = 0
    element_text_string = []
    for element in elements:
        
        if(element_num == 0):
            try:
                text = element.text

                # Check if the element contains both "$" signs and "-" characters
                if "$" in text and "-" in text:
                    #print('element number ',element_num, text)
                    element_num += 1
                    element_text_string.append(text)

            except Exception as e:
                print(f"Error retrieving text: {e}")

    driver.quit()

In [ ]:
#rest of the shit
if element_text_string:
    split_by_line_elements = element_text_string[0].split('\n')

    correct_elements_indexes_array = []

    for i in range(0, len(split_by_line_elements)):
        if split_by_line_elements[i].startswith('20') and "$" in split_by_line_elements[i]:

            character_to_split_on = '$'

            correct_element = split_by_line_elements[i]
            correct_elements_indexes_array.append(i)

            #print(i, correct_element)
        elif split_by_line_elements[i].startswith('20') and "€" in split_by_line_elements[i]:

            character_to_split_on = '€'

            correct_element = split_by_line_elements[i]
            correct_elements_indexes_array.append(i)
            
        elif split_by_line_elements[i].startswith('20') and "£" in split_by_line_elements[i]:

            character_to_split_on = '£'

            correct_element = split_by_line_elements[i]
            correct_elements_indexes_array.append(i)

    rows_list = []

    for j in range(0, len(correct_elements_indexes_array)):
        this_index = correct_elements_indexes_array[j]
        correct_element = split_by_line_elements[this_index]

        this_row_year = correct_element.split(' ')[0]

        correctelement_without_year = correct_element.split(this_row_year)[1][1:]
        split_string = correctelement_without_year.split(character_to_split_on) #'$'

        try:
            # Extract the numerical values and remove commas
            number_1 = split_string[1].replace(',', '').strip()
            number_2 = split_string[2].replace(',', '').strip()
            number_3 = split_string[3].split('-')[0].replace(',', '').strip()

            # Extract the team
            team = split_string[3].split('-')[1].strip()

            # Append row to the list
            rows_list.append([number_1, number_2, number_3, team, this_row_year])

        except IndexError:
            print(j, "Skipping row due to IndexError.")

    # Create a DataFrame
    columns = ["Weekly Salary", "Yearly Salary", "Inflation-Adjusted Yearly Salary", "Team", "Year"]
    df = pd.DataFrame(rows_list, columns=columns)

    df['Year'] = df['Year'].apply(convert_year_to_date)
    columns_to_convert = ['Weekly Salary', 'Yearly Salary', 'Inflation-Adjusted Yearly Salary']

    df[columns_to_convert] = df[columns_to_convert].astype(int)

    # Apply the multiplication based on the character_to_split_on
    df[columns_to_convert] = df[columns_to_convert].apply(lambda x: multiply_by_factor(x, character_to_split_on))

    if(df['Weekly Salary'].sum() == 0):
        impute_required_salary = True

    else:


        closest_date_row = find_closest_date(df, input_year_test)

else:
    print('Unable to retrieve text after multiple attempts.')


df

## Selenium VALUE Lookup

### Wrapper function

In [ ]:
# TESTING WRAPPER FXN
#NDIAYE CASE
#selenium_lookup_name = transfermarkt_filtered_result_AAA

#ZELARAYAN CASE
# selenium_lookup_name = name_match[2]

#YEMEN case 
name_match = [0, 0, 'Ala Addin Mahdi']
natl_test = 'Yemen'

selenium_lookup_name = name_match[2]

selenium_URL, transfermarkt_boolean = getSeleniumURL(selenium_lookup_name, natl_test, input_year_test) ####add input_date_match
#boolean tells you if the first link found was from transfermarkt

if(transfermarkt_boolean == True):
    market_value_graph_html = new_seleniumFindMarketValueGraph(selenium_URL)

    if(market_value_graph_html == ""):
        #then it didn't work
        0==0
    else:
        axis_points_x, axis_points_y = extract_axis_points(market_value_graph_html)

        data_points_graph = extract_data_points(market_value_graph_html)

        x_points_df = create_x_points_dataframe(axis_points_x)

        y_points_df = create_y_points_dataframe(axis_points_y)

        data_points_table_df = create_data_points_table(data_points_graph)

        data_points_table_df_updated = add_date_difference(data_points_table_df, x_points_df, y_points_df)

        #input_match_date_player

        #findMarketValueFromTable

        result_value_time_of_match = findMarketValueFromTable(data_points_table_df_updated, input_year_test)

        print(result_value_time_of_match)
else:
    0==0
    print('yemen case is working')
    #boolean was false

### Piece by piece

#### Get selenium URL

In [ ]:
#debug getSeleniumURL

#for Ndiaye case use transfermarkt_filtered_result_AAA
#NDiaye case
#search = f"{transfermarkt_filtered_result_AAA} {natl_test} transfermarkt"


#Zelarayan case
#search = f"{name_match[2]} {natl_test} transfermarkt"
#search = f"Zelarayan armenia transfermarkt"

#YEMEN CASE 
#search = f'{name_match[2]} {natl_test} transfermarkt'

###NEW### UNFORESEEN ERROR 3
#search = f'Mudir Al Radaei transfermarkt'

#SAUDI CASE
#search = f'{name_match[2]} {natl_test} transfermarkt'
#search = 'Neymar brazil transfermarkt'
#search = 'Martin Skrtel transfermarkt'


#HUSAIN ALI PELE CASE   
# name_match = [0, 0, 'Husain Ali Pele'] 
# natl_test = 'Bahrain'
# input_year_test = '08.10.15'

#HESHAM NAYEM CASE
# name_match = [0, 0, "Hesham Nayem"]
# natl_test = 'Bahrain'
# input_year_test = "24.03.16"

#DJEPAROV CASE
# name_match = [0, 0, "S. Djeparov"]
# natl_test = 'Uzbekistan'
# input_year_test = "03.09.2015"

#replace name_match[2] with input_name_for_lookup
#replace input_year_test with input_date_of_match
#replace natl_test with input_nationality

#MOHAMED ABU WARDA CASE
# name_match = [0, 0, "Mahmoud Abu Warda"]
# natl_test = 'Palestine'
# input_year_test = "14-11-2019"
# search = f'{name_match[2]} {natl_test} transfermarkt' #replace this
#print(f'google search = {search}')

#MUDIR AL RADAEI CASE 
search = f'{name_match[2]} {natl_test} transfermarkt' #replace this


url = 'https://www.google.com/search'

headers = {'Accept': '*/*', 'Accept-Language': 'en-US,en;q=0.5', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82'}

parameters = {'q': search}

content = requests.get(url, headers=headers, params=parameters).text
soup = BeautifulSoup(content, 'html.parser')

search = soup.find(id='search')
first_link = search.find('a')

url_tosplit = first_link['href']

# Use urlsplit to extract the domain
parsed_url = urlsplit(url_tosplit)

link_invalid_bool = False

if parsed_url.netloc == 'www.transfermarkt.us' or parsed_url.netloc == "www.transfermarkt.com":

    #print('1')
    ###CHECK IF URL IS VALID HERE###
    ###IF NOT DO SOMETHING ELSE
    ###IF YES JUST RETURN IT
    if is_valid_link(url_tosplit):
        print("The link is valid.")
        #ANOTHER NAME CHECK HERE

        if(is_trainer_url_not_player_url(first_link['href']) == True):
            #URL IS A MANAGER URL
            print('this is a manager URL', url_tosplit)
            
            search = f'{name_match[2]} {natl_test} spieler transfermarkt' #replace this

            url = 'https://www.google.com/search'

            headers = {'Accept': '*/*', 'Accept-Language': 'en-US,en;q=0.5', 'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82'}

            parameters = {'q': search}

            content = requests.get(url, headers=headers, params=parameters).text
            soup = BeautifulSoup(content, 'html.parser')

            search = soup.find(id='search')
            first_link = search.find('a')

            url_tosplit = first_link['href']
            if is_valid_link(url_tosplit):
                print("The link is valid.")
                link_invalid_bool = True
                url_is_transfermarkt = True
                name_for_final_url = url_tosplit.split('/')[3]
                code_for_final_url = url_tosplit.split('/')[6]
                url_tosplit = f"https://www.transfermarkt.us/{name_for_final_url}/marktwertverlauf/spieler/{code_for_final_url}"
            else:
                print("The link is not valid.")
                url_tosplit = ""


        else:
            #URL IS A PLAYER URL
            url_is_transfermarkt = True
            link_invalid_bool = True

            name_for_final_url = url_tosplit.split('/')[3]
            code_for_final_url = url_tosplit.split('/')[6]
            url_tosplit = f"https://www.transfermarkt.us/{name_for_final_url}/marktwertverlauf/spieler/{code_for_final_url}"
        #print('yes')
    else:
        print("The link is not valid.")
        url_tosplit = ""
    
else:
    
    url_is_transfermarkt = False #may need to comment this out 

    names_on_page = findNamesOnPageUsingSoup(soup)
    all_names = set(names_on_page['Name'])
    candidate_name_links_search = name_match[2] #replace this

    closest_match_result = closest_match(candidate_name_links_search, all_names)

    if(closest_match_result == None):
        print('no closest match result for:', name_match[2]) #replace this
        ###IMPUTE HERE VALUES
        #DOES THIS JUST RETURN THO? MAYBE YOU TAKE CARE OF IT LATER 
    else:
        #print(closest_match_result)
        print(f'link not from transfermarkt. link is from {parsed_url.netloc}')

        dataframe_filtered = names_on_page[names_on_page['Name'] == closest_match_result].reset_index()

        url_is_transfermarkt = True

        correct_link_for_name = dataframe_filtered.at[0, 'Link']
        print('2')
        if is_valid_link(correct_link_for_name):
            print("The link is valid.")
            name_for_final_url = correct_link_for_name.split('/')[3]
            code_for_final_url = correct_link_for_name.split('/')[6]
            url_tosplit = f"https://www.transfermarkt.us/{name_for_final_url}/marktwertverlauf/spieler/{code_for_final_url}" 
            link_invalid_bool = True
        else:
            print("The link is not valid.")
            url_tosplit = ""
            #link_invalid_bool = True

if(link_invalid_bool == False):
    names_on_page = findNamesOnPageUsingSoup(soup)
    df = remove_row_by_link(names_on_page, first_link['href']).reset_index().drop(columns='index', axis=1)
    df[['Name URL style', 'Spieler']] = df['Link'].apply(extract_information_from_link).apply(pd.Series)

    df = df.drop_duplicates(subset=['Name URL style', 'Spieler'], keep='first').reset_index().drop(columns='index', axis=1)

    result_array_aftertest = []
    players_pagesoup_dictionary = {}

    for i in range(0, len(df)):
        this_url_name = df.at[i, 'Name URL style']
        this_code_name = df.at[i, 'Spieler']
        find_national_team_history_URL = f"https://www.transfermarkt.us/{this_url_name}/nationalmannschaft/spieler/{this_code_name}"

        print(df.at[i, 'Name'], find_national_team_history_URL)

        page_soup_history_pg = grab_transfer_pagesoup(find_national_team_history_URL)

        if(find_national_team_in_player_history(page_soup_history_pg, natl_test) == True):
            result_array_aftertest.append(df.at[i, 'Name'])
            players_pagesoup_dictionary[df.at[i, 'Name']] = page_soup_history_pg


    #need to get the page soup from a guy (national page), then you can run it 

    #get the page soup
    filtered_names = filter_similar_names(name_match[2], result_array_aftertest)
    #for each of these remaining names

    #loop through the dictionary that you made 
    #use the names as keys to get their page soup 
    #do the "find match date in history" function with the page_soup_history_pg and the input_year_test

    date_match_array_players = []

    for i in range(0, len(filtered_names)):
        #print(filtered_names[i], players_pagesoup_dictionary[filtered_names[i]])
        if(find_match_date_in_player_history(input_year_test, players_pagesoup_dictionary[filtered_names[i]]) == True):
            print('true', filtered_names[i])
            ###when it works###
            date_match_array_players.append(filtered_names[i])
        else:
            print('match date didn\'t match for', filtered_names[i])
            

    if(len(date_match_array_players) == 1):
        candidate_name_r6 = date_match_array_players[0]
        print('match found. you never accounted for this!!')
        #you have to return this guy's profile URL 

    elif(len(date_match_array_players) >= 2):
        print('multiple matches', date_match_array_players)
    else:
        print('0 matches', filtered_names)
        url_tosplit = ""
        #wtf is going on here 
        ###IMPUTE HERE - VALUES###
else:
    0==0
#return url_tosplit, url_is_transfermarkt
        

#Zelarayan case
#url_tosplit = 'https://www.transfermarkt.us/lucas-zelarayan/marktwertverlauf/spieler/230030'

#NDiaye case
#url_tosplit = "https://www.transfermarkt.us/mamoutou-ndiaye/marktwertverlauf/spieler/94298"

#names_on_page
url_tosplit


#### Find names on page, using soup

In [ ]:
#DEBUG findNamesOnPageUsingSoup

search_results = soup.find_all('a')

transfermarkt_links = []

for result in search_results:
    link = result.get('href')
    if link:
        if ('transfermarkt' in link) and ('spieler' in link):
            transfermarkt_links.append(link)

#def extract_domain_substring(url):
phrases = ['.us/', '.tr/', '.in/', '.com/', '.uk/', '.de/', '.fr/']

#url = transfermarkt_links[3]


#parsed_url = urlparse(url)
#path = parsed_url.path

names_in_links = []
link_indexes = []
original_links = []

names_dataframe = pd.DataFrame()

rows_counter = 0

for i in range(0, len(transfermarkt_links)):
    url = transfermarkt_links[i]

    for phrase in phrases:
        if phrase in url:

            #print(i, url)
            #original_links.append(url)
            #link_indexes.append(i)
            start_index = url.find(phrase) + len(phrase)
            end_index = url.find('/', start_index)
            if end_index != -1:
                correct_name_from_url = url[start_index:end_index]
                #names_in_links.append(correct_name_from_url)
                #print(i, correct_name_from_url)
                new_row = {'Link': url, 'Name': correct_name_from_url}

                new_row  = pd.DataFrame(new_row, index=[rows_counter])

                # Add the new row to the DataFrame
                names_dataframe = pd.concat([names_dataframe, new_row], ignore_index=True) 

                rows_counter += 1
                
            else:
                print('2', url[start_index:])

filtered_dataframe_names = names_dataframe[names_dataframe['Name'].apply(lambda x: bool(re.match("^[a-zA-Z-]+$", x)))].reset_index().drop(columns='index', axis=1)

filtered_dataframe_names['Name'] = filtered_dataframe_names['Name'].apply(lambda x: x.replace('-', ' '))

filtered_dataframe_names['Name'] = filtered_dataframe_names['Name'].apply(lambda x: x.title())

for i in range(len(filtered_dataframe_names['Link'])):
    link = filtered_dataframe_names.at[i, 'Link']
    occurrences = link.count('https:')
    
    # If there are more than 1 "https:", keep only the part starting from the second occurrence
    if occurrences > 1:
        second_occurrence_index = link.find('https:', link.find('https:') + 1)
        filtered_dataframe_names.at[i, 'Link'] = link[second_occurrence_index:]

filtered_dataframe_names

#### Find Market value graph

In [ ]:
#debug new_seleniumFindMarketValueGraph

max_retries = 5
for retry in range(max_retries):
    service = Service()
    options = webdriver.ChromeOptions()
    driver = webdriver.Chrome(service=service, options=options)

    try:
        ID = "id"
        NAME = "name"
        XPATH = "xpath"
        LINK_TEXT = "link text"
        PARTIAL_LINK_TEXT = "partial link text"
        TAG_NAME = "tag name"
        CLASS_NAME = "class name"
        CSS_SELECTOR = "css selector"

        driver.get(url_tosplit)

        elements = driver.find_elements(By.XPATH, '/html/body/div/main/div[3]/div[1]/div/tm-market-value-development-graph-extended/div/div')

        # Check if elements were found
        if elements:
            # Access the first element in the list
            first_element = elements[0]

            # Get the text content of the element
            element_text = first_element.text
            #print(f"Text Content: {element_text}")

            # Alternatively, get the outer HTML of the element
            element_html = first_element.get_attribute('outerHTML')

            if element_html == '<div class="content-box-headline">Loading...</div>':
                print("Retrying...")
                raise ValueError("Loading... message detected")
            else:
                driver.quit()
                break

        else:
            elements = driver.find_elements(By.XPATH, '/html/body/div/main/div[2]/div[1]/div/tm-market-value-development-graph-extended/div/div')
            if elements:
                # Access the first element in the list
                first_element = elements[0]

                # Get the text content of the element
                element_text = first_element.text
                #print(f"Text Content: {element_text}")

                # Alternatively, get the outer HTML of the element
                element_html = first_element.get_attribute('outerHTML')

                if element_html == '<div class="content-box-headline">Loading...</div>':
                    print("Retrying...")
                    raise ValueError("Loading... message detected")
                else:
                    driver.quit()
                    break

            else:
                print('nayem case')
                element_html = ""

    except ValueError as e:
        print(e)
        driver.quit()
        continue
    else:
        # Close the WebDriver if successful
        driver.quit()
        if len(element_html) == 0:
            # It didn't return anything
            print ("")
        else:
            # print(f"Outer HTML: {element_html}")
            print( element_html)

# If max retries reached and still not successful, return an empty string
print ("")



#### Extract Axis Points (from graph)

In [ ]:
#debug extract_axis_points  
outer_html = element_html #found the -1 error with "<div class=\"content-box-headline\">Loading...</div>"

soup = BeautifulSoup(outer_html, 'html.parser')

# Extract X axis points
x_axis_points = []
x_axis_elements = soup.select('.axis.svelte-oklk3z text')

# Add the origin point
x_axis_points.append({'value': '0', 'coordinates': (0, 320)})

# Extract other X axis points
for i, element in enumerate(x_axis_elements):
    value = element.get_text().strip()
    transform_attribute = element.find_parent('g')['transform']
    x_coordinate = float(transform_attribute.split('(')[1].split(',')[0])
    y_coordinate = float(transform_attribute.split(',')[1].split(')')[0])
    x_axis_points.append({'value': value, 'coordinates': (x_coordinate, y_coordinate)})

# Adjust the last X axis point
#CHECK BEFORE U DO THIS
#x_axis_points[-1]['coordinates'] = (0, 42.727272727272734)

# Calculate coordinates for other X axis points
# x_interval = 55.4545454545
# for i in range(len(x_axis_points) - 2, 0, -1):
#     x_axis_points[i]['coordinates'] = (0, x_axis_points[i+1]['coordinates'][1] + x_interval)

# Extract Y axis points
y_axis_points = []
y_axis_elements = soup.select('.axis.svelte-3ta12v text')
for element in y_axis_elements:
    value = element.get_text().strip()
    transform_attribute = element.find_parent('g')['transform']
    x_coordinate = float(transform_attribute.split('(')[1].split(',')[0])
    y_coordinate = float(transform_attribute.split(',')[1].split(')')[0])
    y_axis_points.append({'value': value, 'coordinates': (x_coordinate, y_coordinate)})

###Y AXIS THING TO COMMENT OUT###
#y_axis_points.append({'value': 2023, 'coordinates': (701.634619143, 0.0)})


if str(x_axis_points[-1]['coordinates'][1]).startswith('88'):
    #print("The value starts with '88'")
    for i in range(0, len(x_axis_points)):
        differencearray = []
        if(i != 0):
            currnumber = x_axis_points[i]['coordinates'][1]
            prevnumber = x_axis_points[i-1]['coordinates'][1]
            difference = prevnumber - currnumber
            differencearray.append(difference)
            #print(i, currnumber, difference)

    coordinate_tickmark_difference = np.average(differencearray)

    x_axis_points.append({'value': '0', 'coordinates': (0, x_axis_points[-1]['coordinates'][1] - coordinate_tickmark_difference)})
else:
    print('no 88 problem')
x_axis_points #y_axis_points

In [ ]:
#TEST extract_axis_points

# Example usage:
outer_html = element_html #found the -1 error with "<div class=\"content-box-headline\">Loading...</div>"
x_points, y_points = extract_axis_points(outer_html)

print("X axis points:")
for point in x_points:
    print(f"{point['value']}: {point['coordinates']}")

print("\nY axis points:")
for point in y_points:
    print(f"{point['value']}: {point['coordinates']}")

#### Extract DATA Points 

In [ ]:
# TEST extract_data_points

# Your HTML string
html_string = outer_html #element_html IS FOR when u don't run outer_html = element_html #WAS outer_html BEFORE

# Call the function
points = extract_data_points(html_string)

# Print the result
for point in points:
    print(point)

#### Create X Points Dataframe

In [ ]:
#TEST create_x_points_dataframe

# Your data
data = x_points #x_axis_points for debugging  #x_points otherwise

# Call the function with your data
x_points_dataframe = create_x_points_dataframe(data)
if(x_points_dataframe.iloc[len(x_points_dataframe) - 1]['Value'] == 0):
    row_value_difference_array = []
    for i in range(len(x_points_dataframe)):
        this_row_value = x_points_dataframe.at[i, 'Value']
        if(i == len(x_points_dataframe) - 1):
            0==0
        elif(i != 0):
            difference_row_to_previous = this_row_value - x_points_dataframe.at[i-1, 'Value']
            row_value_difference_array.append(difference_row_to_previous)

    value_difference_between_rows = np.average(row_value_difference_array)

    x_points_dataframe.at[len(x_points_dataframe)-1, 'Value'] = x_points_dataframe.at[len(x_points_dataframe)-2, 'Value'] + value_difference_between_rows

x_points_dataframe

#### Create Y points dataframe

In [ ]:
#debug create_y_points_dataframe 


data = y_points #use y_points_data during a debug ???

x_list, y_list, date_list = [], [], []

# Iterate through data and fill lists
for entry in data:
    print(entry)
    x, y = entry['coordinates']
    x_list.append(x)
    y_list.append(y)
    # Example usage:
    year, month, has_month = parse_date(entry['value'])

    if(has_month):
        last_day = calendar.monthrange(int(f'20{year}'), list(calendar.month_abbr).index(month))[1]

        
        date_str_withmonth = f"{month} {last_day}, {int(year):02d}"
        date_month_included = datetime.strptime(date_str_withmonth, "%b %d, %y")
        #print(date_month_included)
        date_list.append(date_month_included)
    else:

        
        date_str = f"Dec 31, {int(year):02d}"
        date = datetime.strptime(date_str, "%b %d, %y")
        
        date_list.append(date)

# Create DataFrame
df = pd.DataFrame({'X': x_list, 'Y': y_list, 'Date': date_list})

df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

###UNFORESEEN 4###
result = calculate_differences_date_X_coordinate(df)

last_row_length = df.at[len(df)-1, 'X'] + result[0]

original_timestamp = df.at[len(df)-1, 'Date']
days_to_add = result[1] 

new_timestamp = original_timestamp + pd.to_timedelta(days_to_add, unit='D')

# New row data
new_row = {'X': last_row_length, 'Y': 0, 'Date': new_timestamp}

new_row  = pd.DataFrame(new_row, index=[len(df)])

# Add the new row to the DataFrame
df = pd.concat([df, new_row], ignore_index=True) 

df

In [ ]:
#TEST create_y_points_dataframe

# Your data
y_points_data = y_points #y_points when not doing ndiaye debug #y_axis_points otherwise

# Call the function with your data
y_points_dataframe = create_y_points_dataframe(y_points_data)

#put this below line inside the function (see above)
#y_points_dataframe['Date'] = pd.to_datetime(y_points_dataframe['Date'], errors='coerce')

y_points_dataframe

#### Create Data Points Dataframe

In [ ]:
#TEST create_data_points_table

# Your data
data_points = points

# Call the function with your data
data_points_table = create_data_points_table(data_points)
data_points_table

#### Estimate Market Value (using axis points)

In [ ]:
#TEST estimate_value

# Example usage:
input_y_mv = 306.136364	


estimated_value = estimate_value(input_y_mv, x_points_dataframe)
estimated_value

#### Estimate date (using axis points)

In [ ]:
#TEST estimate_date

# Example usage:
input_x_date = 131.3802231 #531.0777369356792 for zelarayan
#y_points_dataframe

estimated_date = estimate_date(input_x_date, y_points_dataframe)
estimated_date

#### Add date difference column (from match date)

In [ ]:
#TEST add_date_difference
for i in range(0, len(data_points_table)):
    # Handling MV
    this_row_mv_coordinate = data_points_table.at[i, 'Y']
    this_row_mv = estimate_value(this_row_mv_coordinate, x_points_dataframe)
    data_points_table.at[i, 'Market Value'] = this_row_mv

    # Handling Date
    this_row_date_coordinate = data_points_table.at[i, 'X']
    this_row_date = estimate_date(this_row_date_coordinate, y_points_dataframe)
    data_points_table.at[i, 'Date'] = pd.Timestamp(this_row_date)

# Set appropriate data types
data_points_table['Market Value'] = data_points_table['Market Value'].astype(float)
data_points_table['Date'] = pd.to_datetime(data_points_table['Date'], errors='coerce')

data_points_table

#this is for Ndiaye
#do it for zelarayan 



#### Find closest market value (from table)

In [ ]:
#TEST findMarketValueFromTable

# Example usage:

result = findMarketValueFromTable(data_points_table, input_year_test)
print(f"The market value closest to {input_year_test} is: {result}")
    

## Multi Name Match Date Lookup (Dejan Lovren)

In [ ]:
#Start

#search = f"{vet_for_match_season[0]} Transfermarkt"

search = f"Dejan Lovren Transfermarkt"
input_year_test = '08.06.19'

url = 'https://www.google.com/search'

headers = {'Accept' : '*/*', 'Accept-Language': 'en-US,en;q=0.5','User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82',
}
parameters = {'q': search}

content = requests.get(url, headers = headers, params = parameters).text
soup = BeautifulSoup(content, 'html.parser')

search = soup.find(id = 'search')
first_link = search.find('a')

url_tosplit = (first_link['href'])

player_name_string = url_tosplit.split('/')[3]
player_code = url_tosplit.split('/')[6]

find_national_team_history_URL = f"https://www.transfermarkt.us/{player_name_string}/nationalmannschaft/spieler/{player_code}"

page_soup_history_pg = grab_transfer_pagesoup(find_national_team_history_URL)

In [ ]:
#Another example - Server Djeparov?
#djeparov_soup = players_pagesoup_dictionary[filtered_names[i]]
# input_year_test = '03-09-2015' #Djeparov match date
# page_soup_history_pg = djeparov_soup

In [ ]:
#Find name in national team history
boxes = page_soup_history_pg.find_all("div", {"class": "box"})

nationalteamcareer_table = boxes[0].find("table")

rows = nationalteamcareer_table.find_all('tr')

teams_played_for_array = []

for i in range(0, len(rows)): 
    if(i == 0):
        0==0
    elif(i % 2 != 0):
        0==0
    else:
        this_row =  rows[i]

        nat_team_name = this_row.find_all("td", {"class": "hauptlink no-border-links hide-for-small"})[0].text.strip().encode().decode("utf-8")
        teams_played_for_array.append(nat_team_name)

teams_played_for_array

In [ ]:
#FINDING MATCH DATE IN THEIR HISTORY
#datestring_lookup = input_year_test.replace(".", "/")

#month, day, year = datestring_lookup.split('/')
# Swap the month and day
#swapped_date = f'{day}/{month}/{year}'

#swapped_date

correctly_formatted_date = process_date_format_for_transfermarkt_lookup(input_year_test)

#boxes[2].find("table")#.find_all("tr", {"class": "bg_gelb_20"})

#.find_all("div", {"class": "responsive-table"})
table_test = page_soup_history_pg.find_all("div", {"class": "responsive-table"})[1].find_all("tbody")[0]

for i in range(0, len(table_test.find_all('tr'))):
    this_tr_row = table_test.find_all('tr')[i]

    data_row = this_tr_row.find_all("td", {"class": "zentriert"})

    if(len(data_row) == 1):
        0==0
    elif(len(data_row) == 7):
        0==0
        #wasn't in the squad
    elif(len(data_row) == 12):

        match_date_row = data_row[1].text.strip()
        #print(match_date_row)
        if((match_date_row == correctly_formatted_date)):  #remember this was swapped_date
            print(i, match_date_row)
        elif(add_leading_zeros(match_date_row) == correctly_formatted_date): #remember this was swapped_date
            print(i, add_leading_zeros(match_date_row))
    else:
        print(i, len(data_row))

switched_date = datetime.strptime(correctly_formatted_date, "%m/%d/%y").strftime("%d/%m/%y")

for i in range(0, len(table_test.find_all('tr'))):
    this_tr_row = table_test.find_all('tr')[i]

    data_row = this_tr_row.find_all("td", {"class": "zentriert"})

    if(len(data_row) == 1):
        0==0
    elif(len(data_row) == 7):
        0==0
        #wasn't in the squad
    elif(len(data_row) == 12):

        match_date_row = data_row[1].text.strip()
        #print(match_date_row)
        if((match_date_row == switched_date)):  #remember this was swapped_date
            print(i, match_date_row)
        elif(add_leading_zeros(match_date_row) == switched_date): #remember this was swapped_date
            print(i, add_leading_zeros(match_date_row))
    else:
        print(i, len(data_row))


## Lookup Name

In [ ]:
#lookup name
# example_problem = '6. Lovren'
# natl_test = 'Croatia'
# input_year_test = '08.06.19'

example_problem = '19 B. Traoré'
natl_test = 'Burkina Faso'
input_year_test = "07.10.16"
salaries_or_values = 'salary'


if(salaries_or_values == "salary"):
    database_name = leagues_salary
    money_column_name = "Inflation-Adjusted Yearly Salary"
elif(salaries_or_values == "value"):
    database_name = leagues_value
    money_column_name = "Market Value"

    
candidate_name = ""
candidates_set = []
match_type = ""


#players from their country 
dataset_nationality = database_name[database_name['Nationality'] == f"{natl_test}"]['Name'].unique()

if(is_cyrillic(example_problem)):
    #change from cyrillic to english
    example_problem = cyrillic_to_latin(example_problem)

#remove jersey Nums and order initials correctly. 
search_name, final_tokens_name = process_string_newest_ii(example_problem) 

#look their name up in the list of names from their nationality. 
# result = find_closest_string(search_name, dataset_nationality)
result = find_closest_string_newEST(search_name, dataset_nationality, final_tokens_name, example_problem)
if((type(result) == list) & (len(result) >= 2)):
    candidates_set = result
elif(result[0] in dataset_nationality):
    candidate_name = result[0]
elif(result in dataset_nationality):
    print(result)
    #RETURN 
    candidate_name = result
else:
    #no match found after first call 
    print('no initial match found: ', search_name)
    nationality_names_accents_removed = remove_accents_from_strings(dataset_nationality)
    match_accent_accounted = find_closest_string_newEST(search_name, nationality_names_accents_removed,final_tokens_name, example_problem)
    if(match_accent_accounted in nationality_names_accents_removed):
        #print(match_accent_accounted)
        matching_names_with_accents = find_names_with_accents(match_accent_accounted, dataset_nationality)
        if(type(matching_names_with_accents) == str):
            print(matching_names_with_accents)
            #RETURN
            candidate_name = matching_names_with_accents
        elif(len(matching_names_with_accents) == 0):
            print(f'accent-less name found: {match_accent_accounted}. But name not in original dataset')
        else:
            print(f'multiple names found after adding accents: {matching_names_with_accents}')
            candidates_set = matching_names_with_accents


        #MAKE SURE THE NAME WITH ACCENTS IS IN DATASET NATIONALITY 
    else:
        print('no accent match found:', search_name)

        dataset_nationality_backticks = remove_apostrophes_backticks(dataset_nationality) #dataset_nationality_updated
        match_apostrophes_accounted = find_closest_string_newEST(search_name, dataset_nationality_backticks,final_tokens_name, example_problem)
    
        if(match_apostrophes_accounted in dataset_nationality_backticks):

            lastname_match = match_apostrophes_accounted.split()[-1] 
            original_string_nojersey = re.sub(r'^\d+(\.)?\s*', '', example_problem)
            correct_lastname = add_backticks(lastname_match, original_string_nojersey)
            correct_firstname = extract_first_name(match_apostrophes_accounted, lastname_match)
            
            correct_name_full = correct_firstname + ' ' + correct_lastname

            if(correct_name_full in dataset_nationality):
                print(correct_name_full)
                #RETURN
                candidate_name = correct_name_full
            elif(correct_name_full.replace('`', "'") in dataset_nationality):
                print(correct_name_full.replace('`', "'"))
                #RETURN
                candidate_name = correct_name_full.replace('`', "'")
            elif(type(match_apostrophes_accounted) != str):
                print(f'multiple names found after adding backticks: {match_apostrophes_accounted}')
                candidates_set = match_apostrophes_accounted
            else:
                print(f'backtick-less name found: {match_apostrophes_accounted}. But name not in original dataset')
        
        else:
            print('no backtick match found:', search_name)

if(candidate_name != ""):
    match_type = "single"
    print(candidate_name, match_type, search_name)
    #return candidate_name, match_type, search_name
else:
    if(candidates_set != []):
        match_type = "multiple"
        print(candidates_set, match_type, search_name)
        #return candidates_set, match_type, search_name
    else:
        match_type = "none"
        print(0, match_type, search_name)
        #return 0, match_type, search_name
        

### Find closest string

In [ ]:
#find closest string newest


#replace nationality name list with string_list
#replace string_for_search with input_string 
#replace final_tokens with input_final_tokens
#replace input_string with ORIGINAL_NAME_STRING

string_list = dataset_nationality
input_string = search_name
input_final_tokens = final_tokens_name
ORIGINAL_NAME_STRING = example_problem


closest_match = get_close_matches(input_string, string_list, n=1, cutoff=0.8)

closest_match_4 = []
closest_match_3 = []
closest_match_2 = []
index_match = ""
matching_indices = []

if closest_match:
    #TIGHT CLOSE MATCH FUNCTION RETURNS A NAME
    #print('0', closest_match[0], closest_match in string_list)
    #RETURN HERE
    #return closest_match[0]
    print(closest_match[0])
else:
    #Reduce match constraints
    closest_match_ii = get_close_matches(input_string, string_list)
    #Produces a match
    if closest_match_ii:
        if((type(closest_match_ii) == list) & (len(closest_match_ii) >= 2)):
            #Closest match II returns 1 name
            original_string_nojersey = re.sub(r'^\d+(\.)?\s*', '', ORIGINAL_NAME_STRING)
            #Find best match from set of names
            result_1 = find_best_match(closest_match_ii, input_final_tokens, original_string_nojersey)
            if(type(result_1) == list):
                closest_match_ii = result_1
                print(closest_match_ii)
            elif(pd.isna(result_1)):
                #none of the names from closest match ii were a good match
                0==0
            else:
                #1 of the names from closest match ii were a good match
                #print('closest match ii best match: ' + result_1)
                closest_match_ii = result_1
                #RETURN HERE
                #return closest_match_ii
                print(closest_match_ii)
        else:
            #Closest match II returns 1 name
            #print('1 match ' + closest_match_ii[0])
            #RETURN HERE
            #return closest_match_ii[0]
            print(closest_match_ii[0])
    else:
        # no close matches
        last_word = input_string.split()[-1]
        #KREJCI CASE 
        if(last_word == 'Krejčí'):
            match_krejci = get_close_matches(last_word, string_list, n=1, cutoff=0.380952)
            if(type(match_krejci) == list):
                if(len(match_krejci) == 1):
                    match_krejci = match_krejci[0]
                    print(match_krejci)
                    #return match_krejci
                
            
            

        
        #if this is an initial you need to save it as an initial or a word start 
        #if(last_word)

        # Return strings from the list if the last word is in those strings
        matching_strings = [s for s in string_list if last_word in s]

        if matching_strings:
            if(len(matching_strings) == 1):
                print('match string ' + matching_strings[0])
                #RETURN HERE
                #return matching_strings[0]

            else:
                print(matching_strings)
                setofmatches = matching_strings
                
    #elif(closest_match):       
    if(closest_match):
        #RETURN
        print('1', closest_match[0], closest_match[0] in string_list)
    elif(closest_match_ii):
        if(type(closest_match_ii) == str):
            print(closest_match_ii, closest_match_ii in string_list)
        #RETURN
        else:
            print('ii', closest_match_ii)#closest_match_ii[0] in string_list
    elif(closest_match_3):
        #RETURN
        print('3', closest_match_3[0], closest_match_3[0] in string_list)
    elif(closest_match_2):
        #RETURN
        print('2', closest_match_2[0], closest_match_2[0] in string_list)
    elif(closest_match_4):
        #RETURN
        print('4', closest_match_4[0], closest_match_4[0] in string_list)
    elif(index_match != ""):
        #RETURN
        print('end ' + index_match + ORIGINAL_NAME_STRING, matching_indices)
        #return(index_match)
    else:
        #RETURN
        #return("No close match found.")
        print("No close match found.")

### Find best match

In [ ]:
#find best match
closest_match_ii = ['Dejan Lovren', 'Davor Lovren', 'Lovre Kalinic']
input_final_tokens = ['Lovren']
original_string_nojersey = 'Lovren'

#closest_match_ii, input_final_tokens, original_string_nojersey
array = closest_match_ii
final_tokens = input_final_tokens
ORIGINAL_STRING = original_string_nojersey


# Concatenate final tokens to form the expected full name
expected_name = ' '.join(final_tokens)

# Filter names that start with the initial letter
filtered_names = [name for name in array if name.startswith(final_tokens[0])]

if not filtered_names:
    if(len(final_tokens) == 1):
        filtered_names = [name for name in array if name.endswith(final_tokens[-1])]
        final_tokens = [final_tokens[-1]]
    # Try switching the order of final tokens
    else:
        filtered_names = [name for name in array if name.startswith(final_tokens[1])]
        final_tokens = [final_tokens[1], final_tokens[0]]

if not filtered_names:
    print('no matching names found 1')
    #return None  # No matching names found

# Check if the ORIGINAL_STRING contains a backtick/apostrophe
has_backtick_apostrophe = "'" in ORIGINAL_STRING or "`" in ORIGINAL_STRING

# Filter names based on the presence of a backtick/apostrophe
filtered_names = [name for name in filtered_names if "'" in name or "`" in name] if has_backtick_apostrophe else filtered_names

if not filtered_names:
    print('no matching names found 2')
    #return None  # No matching names found

# Calculate Levenshtein distance between the expected name and each remaining name
distances = [Levenshtein.distance(unidecode(expected_name), unidecode(name.replace(" ", ""))) for name in filtered_names]

# Find the minimum distance
min_distance = min(distances)

# Find the indices of names with the minimum distance
min_distance_indices = [i for i, distance in enumerate(distances) if distance == min_distance]

# Return all names with the minimum distance
result_names = [filtered_names[i] for i in min_distance_indices]

# Print or return the result based on your requirement
print(result_names)